In [1]:
!pip install langchain
!pip install datasets
!pip install sentence-transformers
!pip install --upgrade huggingface-hub
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.8/120.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

Importing the modules

In [2]:
import  json
import numpy as np

from transformers import AutoConfig
from typing import Dict
from transformers import AutoTokenizer, AutoModel
import torch
#from opensearchpy import RequestsHttpConnection, AWSV4SignerAuth

from langchain import LLMChain
from langchain.chains import RetrievalQA
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import OpenSearchVectorSearch

Load the Reuters news dataset from the Hugging Face hub

In [3]:
loader = HuggingFaceDatasetLoader("reuters21578",
                                  page_content_column="text",
                                  name="ModLewis")
data = loader.load()
print(len(data))
print(type(data))


/usr/local/lib/python3.10/dist-packages/datasets/load.py:2547: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/6188 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/13625 [00:00<?, ? examples/s]

Generating unused split:   0%|          | 0/722 [00:00<?, ? examples/s]

20535
<class 'list'>


Splitting Only 500 news articles

In [4]:
top_500_articles = data[:500]


Split the news articles into chunks

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=128, chunk_overlap=0)

In [6]:
%%time
docs = splitter.split_documents(top_500_articles)
print(type(docs))
text_list = [doc.page_content for doc in docs]
print(text_list)
# Extract text content from each Document object and join them into a single string
text_string = ' '.join([doc.page_content for doc in docs])
print(type(text_string))

#Converting text_String back to list for similarity search
def split_documents(text):
    # Split the text into individual paragraphs/documents
    documents = text.split("\n REUTER\n")  # Assuming each paragraph is separated by two newline characters
    return documents


# Extract the documents from the list
list = eval(text_string)  # Convert the string representation to a list
# Split the text into individual documents
documents_list = list.split(" REUTER\n")[:-1]  # Splitting based on the "REUTER" tag

print(len(documents_list))




<class 'list'>
['"Yugoslav government plans to stop\\nsubsidising loss-making firms will anger hundreds of thousands\\nof workers, Western', 'diplomats said.\\n    The law, proposed by Prime Minister Branko Mikulic, goes\\ninto effect on July 1 and aims to end a', 'long-standing practice\\nof supporting unprofitable companies. Under the law, wage cuts\\nwill be imposed on losing enterprises,', "while those failing to\\nrecover within a six-month grace period will face liquidation.\\n    The diplomats said Mikulic's", 'attempt to create a market\\neconomy is inevitable, but has still come as a shock to those\\naccustomed to government', 'subsidies.\\n    \\"It was a bitter pill which had to be swallowed, but if an\\noverdose is taken too abruptly, it may have', 'adverse effects on\\nthe system,\\" a Western diplomat said.\\n    He said if the law was applied too strictly it would\\nprobably', 'provoke a new wave of strikes and unrest.\\n    Yugoslavia was swept by strikes last month fol

Instance of model and tokenizer

In [ ]:

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-small-en-v1.5")
model = AutoModel.from_pretrained("BAAI/bge-small-en-v1.5")






Preprocessing the Text for creating Embeddings

In [ ]:
# Define the maximum sequence length expected by the model
max_sequence_length = tokenizer.model_max_length  # Maximum sequence length allowed by the model

# Initialize a list to store the IDs of items exceeding the maximum sequence length
exceeding_length_ids = []

# Check the length of each input item in your dataset
for idx, item in enumerate(documents_list):
    if len(item) > max_sequence_length:
        exceeding_length_ids.append(idx)

# Print the list of IDs
print("IDs of items exceeding the maximum sequence length:")
print((exceeding_length_ids))






def retrieve_text(exceeding_length_ids, documents_list):
    retrieved_texts = []
    for idx in exceeding_length_ids:
        # Retrieve the text for the current index
        text = documents_list[idx]
        retrieved_texts.append(text)
    return retrieved_texts

# Call the retrieve_text function to retrieve the texts
retrieved_texts = retrieve_text(exceeding_length_ids, documents_list)



# Define the maximum chunk size
max_chunk_size = 500

def chunk_and_tokenize_text(text, tokenizer, max_chunk_size):
    # Split the text into chunks of maximum size
    chunks = [text[i:i + max_chunk_size] for i in range(0, len(text), max_chunk_size)]

    # Tokenize each chunk
    tokenized_chunks = [tokenizer(chunk, return_tensors="pt", padding=True, truncation=True) for chunk in chunks]

    return tokenized_chunks


final_token_list = []
def process_texts(retrieved_texts, tokenizer, max_chunk_size):


    # Process each retrieved text
    for text in retrieved_texts:
        # Chunk and tokenize the text
        tokenized_chunks = chunk_and_tokenize_text(text, tokenizer, max_chunk_size)

        # Append the tokenized chunks to the final token list
        final_token_list.append(tokenized_chunks)

    return final_token_list


# Tokenize the retrieved texts and store the tokenized chunks in the final token list
final_token_list = process_texts(retrieved_texts, tokenizer, max_chunk_size)









# Check if the lengths of exceeding_length_ids and item_embeddings_list match
if len(final_token_list) == len(retrieved_texts):
    print("All elements in retrieved_texts have been tokenized and stored in final_token_list.")
else:
    print("There might be missing tokens for elements in retrieved_texts list.")

#create embeddings for each element in final token list

# Initialize an empty list to store embeddings

exceeding_embedding_list = []

# Iterate over each text element in retrieved_texts and its corresponding tokens in final_token_list
for text_tokens in final_token_list:
    # Define a list to store embeddings for each chunk of the current text
    chunk_embeddings = []

    # Iterate over each tokenized chunk of the current text
    for chunk_tokens in text_tokens:
        # Convert tokens to input tensors and generate embeddings for the chunk
        with torch.no_grad():
            outputs = model(**chunk_tokens)

        # Extract the embeddings for the chunk
        chunk_embedding = outputs.last_hidden_state.mean(dim=1)  # Assuming mean pooling
        #print(chunk_embedding.shape)
         # Take the mean pooling over the embeddings for the chunk
        #chunk_embedding_mean = torch.mean(chunk_embedding, dim=1)


        # Append the embeddings for the chunk to the list
        chunk_embeddings.append(chunk_embedding)
        #print(chunk_embeddings)
    # Apply max pooling over the embeddings to obtain a single max-pooled embedding for the entire text Max pooling selects the maximum value for each dimension across the embeddings, which can capture salient features of the text
    text_embedding = torch.max(torch.stack(chunk_embeddings), dim=0).values
    #print(text_embedding.shape)
    # Append the text embedding to the embedding list
    exceeding_embedding_list.append(text_embedding)
    print(exceeding_embedding_list)

# Verify that exceeding_embedding_list has the same length as final_token_list
assert len(exceeding_embedding_list) == len(final_token_list), "Lengths of exceeding_embedding_list and final_token_list don't match"

# Verify that each element in exceeding_embedding_list corresponds to a single text embedding
for embedding in exceeding_embedding_list:
    assert embedding.shape == (1, 384), "Invalid shape for embedding in exceeding_embedding_list"



#for nomal ids below max sequence length
# Initialize a list to store the IDs of items exceeding the maximum sequence length
normal_length_ids = []

# Check the length of each input item in your dataset
for idx, item in enumerate(documents_list):
    if len(item) < max_sequence_length:
        normal_length_ids.append(idx)

def normal_retrieve_text(normal_length_ids, documents_list):
    normal_retrieved_texts = []
    for idx in normal_length_ids:
        # Retrieve the text for the current index
        text = documents_list[idx]
        normal_retrieved_texts.append(text)
    return normal_retrieved_texts

# Call the retrieve_text function to retrieve the texts
normal_retrieved_texts = normal_retrieve_text(normal_length_ids, documents_list)


# Confirm that each element in the embedding_list corresponds to a single text element in the final_token_list
for idx, (text_element, normal_list_ids) in enumerate(zip(normal_retrieved_texts, normal_length_ids), start=1):
    assert len(normal_length_ids) == len(normal_retrieved_texts), f"Lengths of  normal_retrieve_text and normal_length_ids do not match for element {idx}"

# Initialize a list to store the embeddings
normal_embeddings_list = []

# Generate embeddings for each text in normal_retrieved_texts
for text in normal_retrieved_texts:
    # Tokenize the text
    tokens = tokenizer(text, return_tensors="pt", padding=True)

    # Generate embeddings
    with torch.no_grad():
        outputs = model(**tokens)

    # Extract the embeddings
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Mean pooling

    # Append the embeddings to the list
    normal_embeddings_list.append(embeddings)

# Confirm that each element in the normal_embeddings_list corresponds to a single text element in the normal_retrieved_texts list
for embedding, text in zip(normal_embeddings_list, normal_retrieved_texts):
    assert len(normal_retrieved_texts) == len(normal_embeddings_list), "Lengths of embeddings and tokens do not match"

# Verify that each element in normal_embedding_list corresponds to a single text embedding
for embedding in normal_embeddings_list:
    assert embedding.shape == (1, 384), "Invalid shape for embedding in exceeding_embedding_list"





#FINAL EMBEDDING LIST
final_embedding_list= normal_embeddings_list + exceeding_embedding_list

# Verify that each element in final_embedding_list corresponds to a single text embedding
for embedding in final_embedding_list:
    assert embedding.shape == (1, 384), "Invalid shape for embedding in exceeding_embedding_list"




# Check if each embedding in final_embedding_list corresponds to each text in documents_list
for i, (embedding, text) in enumerate(zip(final_embedding_list, documents_list)):
    assert embedding is not None and text is not None, f"Missing embedding or text at index {i}"




# Convert final_embedding_list to a NumPy array
final_embedding_array = np.vstack([embedding.squeeze().numpy() for embedding in final_embedding_list])
assert len(documents_list) == final_embedding_array.shape[0], "Length of document_list and number of rows of final_embedding_array must match"




print(final_embedding_array.shape)









Token IDs that are out of range: []


Creating Fiass L2 index

In [ ]:




import faiss
import numpy as np



#different approach for making fias index Create an index with Faiss different distance metrics
#distance_metrics = [faiss.METRIC_L2, faiss.METRIC_INNER_PRODUCT]
#for metric in distance_metrics:
    #index = faiss.IndexFlat(embeddings_array.shape[1], metric)
#index.add(embeddings_array)


#different approach for making fias index
# Adjusting the number of clusters (nlist) in IVF
#nlist_values = [50, 100, 200]  # Try different values for the number of clusters
#for nlist in nlist_values:
    #index = faiss.IndexIVFFlat(faiss.IndexFlatL2(embeddings_array.shape[1]), embeddings_array.shape[1], nlist)
    #index.train(embeddings_array)
    #index.add(embeddings_array)

# Using IndexFlatL2 for exact L2 distance search

# Assuming final_embedding_list is a list of embeddings
embedding_dim = final_embedding_array.shape[0]
print(embedding_dim)

print("Dimensionality of embeddings:", embedding_dim)
index = faiss.IndexFlatL2(final_embedding_list[0].shape[-1])

# Add embeddings to the index
index.add(final_embedding_array)



129
Dimensionality of embeddings: 129
Distance score: 64.09945678710938
Nearest neighbor 1: NEC Corp &lt;NESI.T> filed suit in Tokyo
District Court to temporarily halt the manufacture and
marketing of &lt;Seiko Epson Corp>'s NEC-compatible personal
computer series, an NEC spokesman said.
    He said Seiko Epson's PC 286 computer and HDD-20 interface
board infringe on software copyrights of the Basic In Out
System program of NEC's best-selling 9801 computer series.
    A spokesman for Seiko Epson, which is part of the Hattori
Seiko Co Ltd &lt;HATT.T> group, said its computer does not
infringe on any copyrights and will be marketed this month.
    The HDD-20 interface board went on sale last December.
 NEC's request for an injunction against Seiko Epson and its
marketing subsidiary is the first entirely domestic lawsuit
charging infringement of computer software copyrights, the
Seiko Epson spokesman said.
    Share market analysts said that NEC controls about 70 pct
of Japan's personal c

Generating Query Embeddings and response

In [ ]:
def generate_query_embedding(query_text):
    # Tokenize the query text data
    querytokenized_input = tokenizer(query_text, return_tensors="pt", padding=True, truncation=True)

    # Pass the tokenized input through the model
    with torch.no_grad():
        outputs = model(**querytokenized_input)

    # Extract the embeddings from the model outputs
    queryembeddings = outputs.last_hidden_state[:, 0]  # Extract [CLS] token embedding from the last layer

    queryembeddings_array = queryembeddings.numpy()

    return queryembeddings_array


def interact():
    while True:
        # Prompt the user for a question
        query_text = input("Enter your question (or type 'quit' to exit): ")

        # Check if the user wants to quit
        if query_text.lower() == "quit":
            print("Exiting...")
            break

        # Convert query text to embedding
        query_embedding = generate_query_embedding(query_text)

        # Perform similarity search
        k = 1  # Number of nearest neighbors to retrieve
        distances, indices = index.search(query_embedding,  k)

        # Retrieve text results corresponding to the nearest neighbors' indices
        nearest_neighbors_text = [documents_list[i] for i in indices[0]]

        # Print the text results of nearest neighbors
        for i, text in enumerate(nearest_neighbors_text):
            print(f"Nearest neighbor {i+1}: {text}")

# Run the interaction loop
interact()


Enter your question (or type 'quit' to exit): NEC controls about Japan's personal computer market
Nearest neighbor 1: South Korea's trade surplus is growing too
fast and the government has started taking steps to slow it
down, Deputy Prime Minister Kim Mahn-je said.
    He told a press conference the government planned to
increase investment, speed up the opening of the local market
to foreign imports and gradually adjust its currency to hold
the surplus "at a proper level."
    But he said the government would not allow the won to
appreciate too much in a short period of time. South Korea has
been under pressure from Washington to revalue the won.
    The U.S. Wants South Korea to cut its trade surplus with
the U.S., Which rose to 7.4 billion dlrs in 1986 from 4.3
billion dlrs in 1985.
    Kim, who is also economic planning minister, said prospects
were bright for the South Korean economy, but the government
would try to hold the current account surplus to around five
billion dlrs a y

Using Sentence Embeddings for Comparison

In [7]:
!pip install sentence-transformers


In [8]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load a pre-trained model
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Initialize an empty list to store embeddings
document_embeddings_list = []

# Iterate through each document in documents_list
for document in documents_list:
    # Generate the embedding for the document
    document_embedding = model.encode(document)

    # Append the embedding to the list
    document_embeddings_list.append(document_embedding)

# Convert the list of embeddings to a NumPy array
document_embeddings_array = np.vstack(document_embeddings_list)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
import faiss

# Function to create Faiss index
def create_index(document_embeddings_array):
    dimension = document_embeddings_array.shape[1]  # Dimension of the embeddings
    index = faiss.IndexFlatL2(dimension)  # L2 distance metric
    index.add(document_embeddings_array)
    return index

# Function to perform similarity search
def search_similar_documents(query_embedding, index, k=5):
    distances, indices = index.search(query_embedding.reshape(1, -1), k)
    return distances, indices

# Initialize Faiss index
index = create_index(document_embeddings_array)

# Loop for user interaction
while True:
    # Get user input
    query = input("Enter your query (type 'quit' to exit): ")

    # Check if user wants to quit
    if query.lower() == 'quit':
        print("Exiting...")
        break

    # Embed the query
    query_embedding = model.encode(query)

    # Perform similarity search
    distances, indices = search_similar_documents(query_embedding, index)

    # Retrieve similar documents
    similar_documents = [documents_list[i] for i in indices[0]]

    # Print similar documents and their distances
    print("Similar documents:")
    for distance, document in zip(distances[0], similar_documents):
        print(f"Distance: {distance}, Document: {document}")


Enter your query (type 'quit' to exit): japan
Similar documents:
Distance: 363.9228515625, Document: The Japanese Finance Ministry said it
plans to hold formal financial talks with France on April 17 in
Tokyo, the second such meeting after a first session in May
last year. The agenda has not yet been decided.
    The Japanese side will be headed by Toyoo Gyohten, Vice
Finance Minister for International Affairs, while the French
delegation will include Daniel Lebegue, Director General of the
Treasury.

Distance: 366.322265625, Document: A top U.S. Official said Japan has little
chance of convincing the U.S. To drop threatened trade
sanctions, despite the efforts of a Japanese team that left for
Washington today.
    Michael Armacost, Under Secretary of State for Political
Affairs, was asked at a press conference whether Japan's moves
to boost its domestic economy and open its markets could
persuade the U.S. Not to impose tariffs on Japanese imports
said, and replied: "...It is probably 